In [1]:
#read dfs# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
import sys 
from sklearn.model_selection import learning_curve
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

In [2]:
train_X=pd.read_csv("../input/vol3-kopya/reduced_train_X.csv")
train_y=pd.read_csv("../input/vol3-kopya/train_y.csv")
normalized_test_df=pd.read_csv("../input/vol3-kopya/reduced_test_X.csv")
df_test1=pd.read_csv("../input/vol3-kopya/df_test1.csv")

In [3]:
train_X=train_X.drop(columns=train_X.columns[0])
train_y=train_y.drop(columns=train_y.columns[0])
normalized_test_df=normalized_test_df.drop(columns=normalized_test_df.columns[0])
df_test1=df_test1.drop(columns=df_test1.columns[0])

In [4]:
# cross-validation with 5 splits
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

#Importing XGBM Classifier 
from xgboost import XGBRegressor

#creating an extreme Gradient boosting instance
xgb = XGBRegressor(random_state=0, tree_method='gpu_hist', nthread=-1, verbosity=0)

# parameters 
parameters = {
                "n_estimators": [100, 300, 500],
                "max_depth": [4, 5, 7],
                "subsample": [0.4, 0.7, 1],
                "reg_lambda":[1, 10, 100],
                "colsample_bytree": [0.4, 0.6, 1]
                
}

In [5]:
# grid search for parameters
grid = RandomizedSearchCV(estimator=xgb,  param_distributions=parameters, cv=cv, n_jobs=-1, n_iter=50)
grid.fit(train_X, train_y)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid.best_params_, grid.best_score_))

# prediction results
y_pred = grid.predict(normalized_test_df)

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


The best parameters are {'subsample': 1, 'reg_lambda': 10, 'n_estimators': 300, 'max_depth': 5, 'colsample_bytree': 0.4} with a score of 0.7230


In [6]:
# prediction results
pred_df=pd.DataFrame(zip(df_test1['sector_id'], y_pred), columns=['sector_id', 'predictions'])

In [7]:
submission_dff=pred_df.groupby('sector_id').sum()
ss=submission_dff.reset_index()

In [8]:
ss=ss.rename(columns={"sector_id": "Sektor_Id", "predictions": "Expected"})

In [9]:
ss.to_csv('out_random_search_xgboost.csv',index=False)
pred_df.to_csv('pred_df_random_search_xg', index=False)